In [1]:
import warnings
warnings.filterwarnings('ignore')

import pickle
from product_classification.data_processing.text_processing import FitCategoricalData, InitFields, CreateDatasets, BuildTextVocabulary, BuildIterators
from product_classification.learner.train import CreateLearner, TrainHighLevels, FinetuneAll, EvaluateClassifier
from product_classification.models import CnnHyperParameters, Epochs, LearningRates

In [2]:
path = "../data"
with open(f"{path}/datasets.pkl", "rb") as handle:
    simple_datasets = pickle.load(handle)
    
with open(f"{path}/multilabel_binarizer.pkl", "rb") as handle:
    multilabel_binarizer = pickle.load(handle)

#### Define CNN Hyperparameters

In [4]:
cnn_hyperparameters = CnnHyperParameters(nb_filters=100,
                                           kernels=[3,4,5],
                                           dropout=0.5,
                                           lrates=LearningRates(init_phase=5e-4,
                                                                finetuning_phase=1e-4),
                                           epochs=Epochs(init_phase=1,
                                                         finetuning_phase=1)
                                          )

#### Init torch components for text processing

In [5]:
fit_categorical_data = FitCategoricalData()
one_hot_encoder = fit_categorical_data.execute(processed_data=simple_datasets)

In [6]:
init_fields = InitFields()
torch_fields = init_fields.execute(one_hot_encoder=one_hot_encoder)

2022-09-11 16:11:23,654 :: text_processing/text_processing.py/execute :: INFO :: Categorical field created
2022-09-11 16:11:24,943 :: text_processing/text_processing.py/execute :: INFO :: Text field created
2022-09-11 16:11:24,944 :: text_processing/text_processing.py/execute :: INFO :: Label field created


#### Create datasets mapping features and labels

In [7]:
labels = list(simple_datasets.training.iloc[:,5:].columns)
cat_cols = ["brand_name", "merchant_name"]
create_datasets = CreateDatasets()
torch_dataset = create_datasets.execute(processed_data=simple_datasets,
                                        torch_fields=torch_fields,
                                        cnn_hparams=cnn_hyperparameters,
                                        txt_col="product_name",
                                        cat_cols=cat_cols,
                                        lbl_cols=labels)

2022-09-11 16:11:24,961 :: __init__/__init__.py/__init__ :: INFO :: fields: [('text', <torchtext.data.field.Field object at 0x7f1752a79e80>), ('category', <torchtext.data.field.Field object at 0x7f1752a79d00>), ('label', <torchtext.data.field.LabelField object at 0x7f17e449e550>)]
2022-09-11 16:17:29,720 :: __init__/__init__.py/__init__ :: INFO :: fields: [('text', <torchtext.data.field.Field object at 0x7f1752a79e80>), ('category', <torchtext.data.field.Field object at 0x7f1752a79d00>), ('label', <torchtext.data.field.LabelField object at 0x7f17e449e550>)]
2022-09-11 16:17:36,751 :: __init__/__init__.py/__init__ :: INFO :: fields: [('text', <torchtext.data.field.Field object at 0x7f1752a79e80>), ('category', <torchtext.data.field.Field object at 0x7f1752a79d00>), ('label', <torchtext.data.field.LabelField object at 0x7f17e449e550>)]
2022-09-11 16:19:04,666 :: text_processing/text_processing.py/execute :: INFO :: Torchtext datasets created from dataframes


#### Build text vocabulary with fasttext embeddings

In [8]:
vector_cache=".vector_cache"
vocab_size=100000
embedding_name="fasttext.simple.300d"
build_text_vocabulary = BuildTextVocabulary()
torch_fields = build_text_vocabulary.execute(torch_datasets=torch_dataset,
        torch_fields=torch_fields,
        vocab_size=vocab_size,
        embedding_name=embedding_name,
        vectors_cache=vector_cache)

2022-09-11 16:19:05,672 :: text_processing/text_processing.py/execute :: INFO :: Text vocabulary created. Corpus_dim: 30851


#### Create pytorch iterators

In [9]:
batch_size=128 
device="cpu"
build_iterators = BuildIterators()
torch_iterators = build_iterators.execute(torch_datasets=torch_dataset,
                                             batch_size=batch_size,
                                             device=device)

2022-09-11 16:19:05,682 :: text_processing/text_processing.py/execute :: INFO :: Torchtext iterators created from datasets


#### Create Learner to train CNN

In [10]:
create_learner = CreateLearner()
learner = create_learner.execute(cnn_hparams=cnn_hyperparameters,
                                    embedding_name=embedding_name,
                                    torch_fields=torch_fields,
                                    processed_data=simple_datasets,
                                    batch_size=batch_size,
                                    label_number=len(labels),
                                    one_hot_encoder=one_hot_encoder)

2022-09-11 16:19:05,878 :: train/train.py/execute :: INFO :: CNN learner compiled for the text classification task


#### Training

Train high levels layers (embedding layer freezed)

In [11]:
train_high_levels = TrainHighLevels()
learner = train_high_levels.execute(cnn_learner=learner,
        device=device,
        torch_iterators=torch_iterators,
        cnn_hparams=cnn_hyperparameters)

2022-09-11 16:19:05,887 :: train/train.py/execute :: INFO :: Starting training of CNN and classification layers


Embedding(30851, 300)


Training Epoch [1/1]:  10%|██████████▋                                                                                                | 39/391 [01:40<40:12,  6.85s/it, loss=0.707]

Validation:
	hamming_loss = 0.031
	f1_score = 0.283
	loss = 0.668
****************************************


Training Epoch [1/1]:  20%|█████████████████████▌                                                                                      | 78/391 [03:19<35:19,  6.77s/it, loss=0.47]

...

Validation:
	hamming_loss = 0.013
	f1_score = 0.657
	loss = 0.042
****************************************


Training Epoch [1/1]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [16:26<00:00,  2.52s/it, loss=0.14]


Training at epoch 1:
	hamming_loss = 0.013
	f1_score = 0.656
	loss = 0.140


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:15<00:00,  1.88s/it, loss=0.0366]

Validation:
	hamming_loss = 0.013
	f1_score = 0.658
	loss = 0.042
****************************************


Train all model layers

In [12]:
finetune_all = FinetuneAll()
learner = finetune_all.execute(cnn_learner=learner,
                                device=device,
                                torch_iterators=torch_iterators,
                                cnn_hparams=cnn_hyperparameters)

2022-09-11 16:35:47,607 :: train/train.py/execute :: INFO :: Unfreezing the whole CNN network
2022-09-11 16:35:47,611 :: train/train.py/execute :: INFO :: Starting finetuning of the whole CNN network
Training Epoch [1/1]:  10%|██████████▋                                                                                                | 39/391 [01:46<40:30,  6.90s/it, loss=0.612]

Validation:
	hamming_loss = 0.083
	f1_score = 0.643
	loss = 0.622
****************************************


Training Epoch [1/1]:  20%|█████████████████████▎                                                                                     | 78/391 [03:32<36:10,  6.93s/it, loss=0.373]

Validation:
	hamming_loss = 0.016
	f1_score = 0.558
	loss = 0.059
****************************************


Training Epoch [1/1]:  30%|███████████████████████████████▋                                                                          | 117/391 [05:16<31:10,  6.83s/it, loss=0.266]

...

Validation:
	hamming_loss = 0.011
	f1_score = 0.741
	loss = 0.035
****************************************


Training Epoch [1/1]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [17:26<00:00,  2.68s/it, loss=0.107]


Training at epoch 1:
	hamming_loss = 0.011
	f1_score = 0.736
	loss = 0.107


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:14<00:00,  1.83s/it, loss=0.0307]
2022-09-11 16:53:28,816 :: train/train.py/execute :: INFO :: Training done


Validation:
	hamming_loss = 0.011
	f1_score = 0.741
	loss = 0.035
****************************************


#### Evaluation

In [13]:
evaluate_classifier = EvaluateClassifier()
perfs = evaluate_classifier.execute(cnn_learner=learner,
                                        device=device,
                                        torch_iterators=torch_iterators,
                                        multilabel_binarizer=multilabel_binarizer)

2022-09-11 16:56:06,153 :: train/train.py/execute :: INFO :: Test results
2022-09-11 16:56:06,156 :: train/train.py/execute :: INFO :: {
    "animalerie": {
        "precision": 0.9841772151898734,
        "recall": 0.4712121212121212,
        "f1-score": 0.6372950819672132,
        "support": 660
    },
    "auto et moto": {
        "precision": 0.0,
        "recall": 0.0,
        "f1-score": 0.0,
        "support": 342
    },
    "bagages et sacs": {
        "precision": 0.6666666666666666,
        "recall": 0.008333333333333333,
        "f1-score": 0.01646090534979424,
        "support": 240
    },
    "beaute et parfum": {
        "precision": 0.9097888675623801,
        "recall": 0.7834710743801653,
        "f1-score": 0.8419182948490231,
        "support": 605
    },
    "bebe et puericulture": {
        "precision": 1.0,
        "recall": 0.15447154471544716,
        "f1-score": 0.2676056338028169,
        "support": 369
    },
    "bijoux": {
        "precision": 0.974358974358